In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D
from keras import backend as K

#data = pd.read_csv('https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/FR_E2_2021-01-01.csv',delimiter=';', encoding='latin-1')

# Problématique: 

Une fois dans l’air, le monoxyde d’azote (NO) devient du dioxyde d’azote (NO2), gaz irritant pour les bronches et favorisant les crises d’asthmes et les infections pulmonaires. Les personnes asthmatiques et les jeunes enfants sont plus sensibles à ce polluant.

#Peut-on alterter ces gens?

#Perspective

Nous proposons dans ce travail une approche LSTM/ IA afin de prédire la valeur de desnité des polluants atmospheriques.

#Analyse des données
Aprés une analyse sur les données nous avons constaté que celui-ci est constitué de 5 regions principales: 

###Périurbaine               9696
###Rurale nationale          1272
###Rurale régionale          2184
###Urbaine                   27744

#Notre objectif:

Predire la de densité des polluants atmosphériques en France, nous avons pensé à extraire un seul type d'implementation 
qui est Urbaine ( dans les villes), afin de mettre en place un modele de deep leraning de type LSTM, pour prédire la densité atmospherique de celui-ci.

#Jeu de données
Le jeu de données represente la concentrations des polluants atmospheriques en France. Provenant de la source suivante: 
https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/FR_E2_2021-01-01.csv 

Les infulences sont représentées par: code, site,	nom site,	type d'implantation,polluant,	type d'influence,	discriminant,	réglementaire,type d'évaluation,procédure de mesure,type de valeur,valeur brute,	unité de mesure,	taux de saisie,	couverture temporelle,couverture de données,code qualité,validité.

L'objectif est d'alerter les gens ayants des problemes de santé, afin d'eviter l'impact probable sur leurs santé.



#Data export

In [ ]:
data = pd.read_csv('https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/FR_E2_2021-01-01.csv',delimiter=';', encoding='latin-1')

In [ ]:
df = data.groupby(['type d\'implantation']).size()
df

type d'implantation
Périurbaine                9696
Rurale nationale           1272
Rurale près des villes     1824
Rurale régionale           2184
Urbaine                   27744
dtype: int64

#Data exploration 

In [ ]:
# Nous allons travailler sur les regions urbaines: car elles sont plus touchées par l'impact de la DPA

In [ ]:
data = data[data['type d\'implantation']=='Urbaine']
data.head()

date de début          date de fin  ... code qualité validité
0  2021/01/01 00:00:00  2021/01/01 01:00:00  ...            A        1
1  2021/01/01 01:00:00  2021/01/01 02:00:00  ...            A        1
2  2021/01/01 02:00:00  2021/01/01 03:00:00  ...            A        1
3  2021/01/01 03:00:00  2021/01/01 04:00:00  ...            A        1
4  2021/01/01 04:00:00  2021/01/01 05:00:00  ...            A        1

[5 rows x 23 columns]

In [ ]:
data.isna().sum()

date de début                0
date de fin                  0
organisme                    0
code zas                     0
zas                          0
code site                    0
nom site                     0
type d'implantation          0
polluant                     0
type d'influence             0
discriminant              4248
réglementaire                0
type d'évaluation            0
procédure de mesure          0
type de valeur               0
valeur                     742
valeur brute               742
unité de mesure              0
taux de saisie           27744
couverture temporelle    27744
couverture de données    27744
code qualité                 0
validité                     0
dtype: int64

In [ ]:
data['discriminant'] = data['discriminant'].replace(np.nan, 0)
data['valeur'] = data['valeur'].replace(np.nan, 0)
data['valeur brute'] = data['valeur brute'].replace(np.nan, 0)
data['taux de saisie'] = data['taux de saisie'].replace(np.nan, 0)
data['couverture temporelle'] = data['couverture temporelle'].replace(np.nan, 0)
data['taux de saisie'] = data['taux de saisie'].replace(np.nan, 0)

In [ ]:
#Drop columns
#data.dropna(subset=['couverture temporelle', 'stop_time'], inplace=True)

In [ ]:
data[data.notnull()]
data['couverture de données'] = data['couverture de données'].replace(np.nan, 0)
data.isna().sum()

date de début            0
date de fin              0
organisme                0
code zas                 0
zas                      0
code site                0
nom site                 0
type d'implantation      0
polluant                 0
type d'influence         0
discriminant             0
réglementaire            0
type d'évaluation        0
procédure de mesure      0
type de valeur           0
valeur                   0
valeur brute             0
unité de mesure          0
taux de saisie           0
couverture temporelle    0
couverture de données    0
code qualité             0
validité                 0
dtype: int64

In [ ]:

dataNew=data.rename(columns={'validité':'validate','valeur brute':'val_brute','procédure de mesure':'proced_mesure','valeur':'valeur','type d\'évaluation':'evaluatedType',
                     'réglementaire':'reglement','type d\'implantation':'implementType','unité de mesure':'mesureUnite','type d\'influence':'influencetype',
                     'couverture de données':'data_couverting','code qualité':'qualityCode','taux de saisie':'SaisieTaux','type de valeur':'ValueType','code site':'siteCode','nom site':'siteName'}, errors="raise")
dataNew.head()

date de début          date de fin  ... qualityCode validate
0  2021/01/01 00:00:00  2021/01/01 01:00:00  ...           A        1
1  2021/01/01 01:00:00  2021/01/01 02:00:00  ...           A        1
2  2021/01/01 02:00:00  2021/01/01 03:00:00  ...           A        1
3  2021/01/01 03:00:00  2021/01/01 04:00:00  ...           A        1
4  2021/01/01 04:00:00  2021/01/01 05:00:00  ...           A        1

[5 rows x 23 columns]

In [ ]:
cat_data = []
num_data = []
for i,c in enumerate(dataNew.dtypes):
  if c == object:
    cat_data.append(dataNew.iloc[:,i])
  else:
    num_data.append(dataNew.iloc[:,i])

cat_data = pd.DataFrame(cat_data).transpose()
num_data = pd.DataFrame(num_data).transpose()

In [ ]:
cat_data.head()

date de début          date de fin  ... mesureUnite qualityCode
0  2021/01/01 00:00:00  2021/01/01 01:00:00  ...       µg-m3           A
1  2021/01/01 01:00:00  2021/01/01 02:00:00  ...       µg-m3           A
2  2021/01/01 02:00:00  2021/01/01 03:00:00  ...       µg-m3           A
3  2021/01/01 03:00:00  2021/01/01 04:00:00  ...       µg-m3           A
4  2021/01/01 04:00:00  2021/01/01 05:00:00  ...       µg-m3           A

[5 rows x 17 columns]

In [ ]:
data.shape

(27744, 23)

In [ ]:
# encoding one hot to cat_data
cat_data = pd.get_dummies(cat_data)
cat_data


date de début_2021/01/01 00:00:00  ...  qualityCode_R
0                                      1  ...              0
1                                      0  ...              0
2                                      0  ...              0
3                                      0  ...              0
4                                      0  ...              0
...                                  ...  ...            ...
42715                                  0  ...              0
42716                                  0  ...              0
42717                                  0  ...              0
42718                                  0  ...              0
42719                                  0  ...              0

[27744 rows x 900 columns]

In [ ]:
#concatenation
data_final = pd.concat([num_data, cat_data], axis = 1) 

In [ ]:
data_final

valeur  val_brute  ...  qualityCode_N  qualityCode_R
0         0.6      0.600  ...              0              0
1         1.0      1.000  ...              0              0
2         0.6      0.625  ...              0              0
3         0.8      0.775  ...              0              0
4         1.3      1.250  ...              0              0
...       ...        ...  ...            ...            ...
42715    18.6     18.625  ...              0              0
42716    19.8     19.775  ...              0              0
42717    26.5     26.450  ...              0              0
42718    19.7     19.725  ...              0              0
42719    16.4     16.425  ...              0              0

[27744 rows x 906 columns]

#Data normalize

In [ ]:
# Normaliser X_train
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
data_final = sc.fit_transform(data_final)

In [ ]:
data_final.shape

(27744, 906)

In [ ]:
#Une analyse Multivarié


#Split data

In [ ]:
train_data = data_final[0:22195]
val_data = data_final[22195:]

#Creation de sequences

In [ ]:
def create_sequences(data, history_size, future_size):
 features = []
 labels = []

 start_index = history_size
 end_index = len(data) - future_size + 1

 for i in range(start_index,end_index):
   features.append(data[i-history_size:i])
   labels.append(data[i:i+future_size])

 return np.array(features), np.array(labels)

In [ ]:
#Chaque 16 jours

In [ ]:
history_size = 16
future_size = 1
x_train, y_train = create_sequences(train_data, history_size, future_size)
x_val, y_val = create_sequences(val_data, 16, 1)

In [ ]:
x_train

array([[[0.01184   , 0.01176282, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.01312   , 0.01304313, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.01184   , 0.01184284, 0.        , ..., 1.        ,
         0.        , 0.        ],
        ...,
        [0.01696   , 0.01688405, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.0176    , 0.01760423, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.01664   , 0.01648396, 0.        , ..., 1.        ,
         0.        , 0.        ]],

       [[0.01312   , 0.01304313, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.01184   , 0.01184284, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.01248   , 0.01232296, 0.        , ..., 1.        ,
         0.        , 0.        ],
        ...,
        [0.0176    , 0.01760423, 0.        , ..., 1.        ,
         0.        , 0.        ],
        [0.0

#Reshaping

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 906)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 906)
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((22179, 16, 906), (22179, 1, 906), (5533, 16, 906), (5533, 1, 906))

#Creation du modele et entrainement

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [ ]:
import keras.models
import tensorflow
# Initialising the RNN

regressor = Sequential()
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 453, return_sequences = True, input_shape = (x_train.shape[1], 906)))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 453))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 906))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

# Fitting the RNN to the Training set
history = regressor.fit(x_train, y_train, epochs = 40 , batch_size = 20,validation_data=(x_val, y_val))


Epoch 1/40
1109/1109 [==============================] - 912s 799ms/step - loss: 0.0118 - accuracy: 0.6017 - val_loss: 0.0113 - val_accuracy: 0.2539
Epoch 2/40
1109/1109 [==============================] - 893s 805ms/step - loss: 0.0113 - accuracy: 0.7541 - val_loss: 0.0112 - val_accuracy: 0.9626
Epoch 3/40
1109/1109 [==============================] - 909s 820ms/step - loss: 0.0112 - accuracy: 0.8063 - val_loss: 0.0112 - val_accuracy: 0.8776
Epoch 4/40
1109/1109 [==============================] - 899s 811ms/step - loss: 0.0112 - accuracy: 0.7735 - val_loss: 0.0112 - val_accuracy: 0.9726
Epoch 5/40
1109/1109 [==============================] - 897s 809ms/step - loss: 0.0112 - accuracy: 0.8431 - val_loss: 0.0113 - val_accuracy: 0.9680
Epoch 6/40
1109/1109 [==============================] - 899s 811ms/step - loss: 0.0112 - accuracy: 0.7631 - val_loss: 0.0112 - val_accuracy: 0.6864
Epoch 7/40
1109/1109 [==============================] - 901s 813ms/step - loss: 0.0112 - accuracy: 0.8164 - val_

In [ ]:
reconstructed_model.summary()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

In [ ]:
for k in history.history.keys(): 
    
    plt.plot(history.history[k])

plt.legend(labels)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15, 6))
plt.plot(np.mean(X_train,axis=0))
plt.plot(np.mean(val_loss, axis=0))

plt.legend(['Training'])

In [ ]:
data.ZZ.value_counts().plot(kind='bar')
plt.show()

In [ ]:
# def rmse(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

# model.compile(optimizer='adam', loss=rmse)
# history = model.fit(x_train, y_train, batch_size=20, epochs=50, validation_data=(x_val, y_val))
# # Score model
# score = model.evaluate(x_val, y_val, verbose=0)
# print('Test loss:', score)
# # Save model
# #model.save(f"models/new_model")

#Graphiques

In [1]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for k in history.history.keys(): 
    
    plt.plot(history.history[k])
plt.legend()

In [ ]:
x_val

In [ ]:
#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

#Prediction 

In [ ]:
# Nous allons maintenant tester notre modele sur des nouvelles données:
x_val[1]

In [ ]:
y_hat = regressor.predict(x_val[2].reshape(1, 16, 906))
y_hat

In [ ]:
#Prediction 
def predict(i):
    ZZ= []
    z = regressor.predict(x_val)
    z
    # si z est supérieur à 0,5, notre prédiction est de signe un (y_head=1),
    # si z est inférieur à 0,5, notre prédiction est de signe zéro (y_head=0),
    for j in z:
        if (j > 0.5) is True:
           j = 1
           ZZ.append(j)
           ZZ
        elif (j < 0.5) is True:
             j = 0
             ZZ.append(j)
             ZZ
    return predict(i)
predict(1)

In [ ]:
y_hat = regressor.predict(x_val[1].reshape(1, 16, 906))
#print("Forecasted value: "+str(y_hat[0][0]))
print("Actual value:     "+str(y_val[2][0]))

In [ ]:
# y_hat = model.predict(x_val[1].reshape(1,16,1))
# print("Forecasted value: "+str(y_hat[0][0]))
# print("Actual value:     "+str(y_val[1][0]))

In [ ]:
# y_hat = model.predict(x_val[1].reshape(1,16,1))
# print("Forecasted value: "+str(y_hat[0][0]))
# print("Actual value:     "+str(y_val[2][0]))